Recognise Streetview Digits
=============


3. Implementation
------------

- 3.1 Import modules and data
- 3.2 Create helper functions
- 3.3 Set up the model
- 3.4 Run the model with different hyperparameters
- 3.5 Show the results

**3.1 Import modules and data**

Get the modules and load the data from the pickle.

In [1]:
from __future__ import print_function
from __future__ import division

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib.font_manager import FontProperties
import seaborn as sns
%matplotlib inline

import numpy as np
import pandas as pd
import tensorflow as tf
import random

import os
from six.moves import cPickle as pickle

from timeit import default_timer as timer
from time import gmtime, strftime
import time

In [2]:
def load_from_pickle(pickle_file):
  with open(pickle_file, 'rb') as f:
    data = pickle.load(f)
  print('Data loaded from %s.' % pickle_file)
  return data

# train_dataset, train_labels = load_from_pickle('train_data_preprocessed.pickle')
# test_dataset,  test_labels  = load_from_pickle('test_data_preprocessed.pickle')
extra_dataset_1, extra_labels_1 = load_from_pickle('extra_data_preprocessed_1_stretch.pickle')
# extra_dataset_2, extra_labels_2 = load_from_pickle('extra_data_preprocessed_2.pickle')
test_photos = load_from_pickle('test_photos.pickle')

# to fit with memory constraints, we only use the first 100k 'extra' images for train and test.
train_size = 85000 
train_dataset, train_labels = extra_dataset_1[:train_size,:,:,:], extra_labels_1[:train_size,:]
test_dataset,  test_labels  = extra_dataset_1[train_size:,:,:,:], extra_labels_1[train_size:,:]
# train_dataset, train_labels = extra_dataset_1[:train_size,:,:], extra_labels_1[:train_size,:]
# test_dataset,  test_labels  = extra_dataset_1[train_size:,:,:], extra_labels_1[train_size:,:]

print('Train data',   train_dataset.shape, train_dataset.dtype)
print('Train labels', train_labels.shape, train_labels.dtype)
print('Test data',    test_dataset.shape, test_dataset.dtype)
print('Test labels',  test_labels.shape, test_labels.dtype)
print('Test photos',  test_photos.shape, test_photos.dtype)

Data loaded from extra_data_preprocessed_1_stretch.pickle.
Data loaded from test_photos.pickle.
Train data (85000, 64, 64, 1) float32
Train labels (85000, 7) float64
Test data (15000, 64, 64, 1) float32
Test labels (15000, 7) float64
Test photos (12, 64, 64) float32


Reformat image data into a TensorFlow convolution-friendly shape (width by height by #channels). 

No need to reformat labels to 1-hot encodings (as we'll use sparse_softmax_cross_entropy_with_logits), but we need to remove the first label (the number length) given the simplified approach we're taking.

In [3]:
# Define global constants
IMAGE_SIZE      = 64
NUM_LABELS      = 11   # the logical size of the one-hots 
NUM_CLASSIFIERS =  5   # the number of softmax classifiers we'll create
NUM_CHANNELS    =  1   # grayscale

def reshape_images(images):  # reshape to have each pixel value in its own array, for the convolutions, if not already done
  images = images.reshape((-1, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS)).astype(np.float32)
  print('Data', images.shape, images.dtype)
  return images

def reshape_labels(labels):  # ignore the length, just include the first n digits, where 0 means "no digit"  
  labels = labels[:, 1:NUM_CLASSIFIERS+1].astype(np.int64)
  print('Labels', labels.shape, labels.dtype)
  print(np.array([np.histogram(label_i, bins=range(11))[0] for label_i in np.transpose(labels)], dtype='int'))
  return labels

print('Train data:')
train_dataset, train_labels = reshape_images(train_dataset), reshape_labels(train_labels)
print('Test data:')
test_dataset, test_labels   = reshape_images(test_dataset),  reshape_labels(test_labels)
print('Test photos:')
test_photos = reshape_images(test_photos)

Train data:
Data (85000, 64, 64, 1) float32
Labels (85000, 5) int64
[[    0 23759 16564 11454  8997  6927  5255  4995  3615  3434]
 [ 3921  8815  9170  8475  7541  8598  7203  7834  6468 16975]
 [34113  4850  5065  4873  4159  6319  4612  5033  4154 11822]
 [78965   596   531   557   511   749   532   583   461  1515]
 [84952     3     1     3     3     4     9     8     3    14]]
Test data:
Data (15000, 64, 64, 1) float32
Labels (15000, 5) int64
[[    0  4193  2998  2083  1570  1237   863   863   605   588]
 [  710  1480  1663  1501  1407  1433  1196  1403  1183  3024]
 [ 5903   863   900   893   737  1109   848   906   733  2108]
 [13880   119    93   100    87   146   106    98    90   281]
 [14988     1     1     1     2     3     0     0     2     2]]
Test photos:
Data (12, 64, 64, 1) float32


**3.2 Create helper functions**

First define the low level functions to create components of layers.

In [4]:
def create_placeholders():
  input_data = tf.placeholder(tf.float32, shape=(None, IMAGE_SIZE, IMAGE_SIZE, NUM_CHANNELS), name='Input/Data')
  labels     = tf.placeholder(tf.int64, shape=(None, NUM_CLASSIFIERS), name='Input/Labels')
  keep_prob  = tf.placeholder(tf.float32, name='Dropout_keep_prob')
  tf.scalar_summary('Parameters/Dropout (keep)',keep_prob)
  return input_data, labels, keep_prob

def add_variable_summaries(variable, name): # To analyse all variables in tensorboard
  if TENSORBOARD_SHOW_ALL:
    with tf.name_scope('Summaries'):
      tf.scalar_summary(name+'/min', tf.reduce_min(variable))
      tf.scalar_summary(name+'/max', tf.reduce_max(variable))
      mean = tf.reduce_mean(variable) # put it in a variable as we use it twice
      with tf.name_scope('Std_dev'):
        std_dev = tf.sqrt(tf.reduce_mean(tf.square(variable - mean)))
      tf.scalar_summary(name+'/mean', mean)
      tf.scalar_summary(name+'/std_dev', std_dev)
      tf.histogram_summary(name, variable)

def create_weights(layer_name, dimensions):
  with tf.name_scope('Weights'):
    input_dims = np.prod(dimensions[:-1])   # the last element is the output dimension, all others inputs
    init_stddev = np.sqrt(2.0 / input_dims) # this ensures the variance after ReLU is 1
    weights = tf.Variable(tf.truncated_normal(dimensions,stddev = init_stddev))
    add_variable_summaries(weights, layer_name+'/weights')
  return weights

def create_biases(layer_name, output_dim):
  with tf.name_scope('Biases'):
    biases  = tf.Variable(tf.zeros([output_dim])) # initialise to zero
    add_variable_summaries(biases, layer_name+'/biases')
  return biases

def create_activations(layer_name, pre_activations, activation_func):
  with tf.name_scope('Activations'):
    activations = activation_func(pre_activations)
    tf.histogram_summary(layer_name+'/2_activations', activations)
  return activations

def add_l2_losses(layer_name, l2_loss_list, weights, biases):
  with tf.name_scope('L2_losses'):
    l2_losses = tf.nn.l2_loss(weights) + tf.nn.l2_loss(biases)
    tf.scalar_summary(layer_name+'/L2_losses',l2_losses)
    l2_loss_list.append(l2_losses)
    return l2_loss_list

Now define the functions to build the layers.

In [5]:
def create_conv_layer(layer_name, input_tensor, l2_loss_in, patch_size, out_channels, stride, 
                      max_pool_kernel, max_pool_stride, activation_func):
  in_channels = input_tensor.get_shape().as_list()[3]
  with tf.name_scope(layer_name):
    weights = create_weights(layer_name, [patch_size, patch_size, in_channels, out_channels])
    biases  = create_biases(layer_name, out_channels)
    with tf.name_scope('Convolutions'):
      strides = [1, stride, stride, 1]
      convolutions = tf.nn.conv2d(input_tensor, weights, strides, padding="SAME")
      tf.histogram_summary(layer_name+'/1_convolutions', convolutions)
    with tf.name_scope('Max_pooling'):
      if max_pool_stride > 1:
        ksize   = [1, max_pool_kernel, max_pool_kernel, 1]
        strides = [1, max_pool_stride, max_pool_stride, 1]
        convolutions = tf.nn.max_pool(convolutions, ksize, strides, padding="SAME")
    activations = create_activations(layer_name, convolutions + biases, activation_func)
    l2_loss_out = add_l2_losses(layer_name, l2_loss_in, weights, biases)
  return activations, l2_loss_out, weights

def create_all_conv_layers(next_input, l2_loss, conv_layers, keep_prob):
  weights_list = [] # return the weights for visualisation
  for i, conv_layer in enumerate(conv_layers):
    patch_size, depth, stride, mp_kernel, mp_stride = conv_layer
    next_input, l2_loss, weights = create_conv_layer('Convolution'+str(i+1), next_input, l2_loss, 
                                            patch_size, depth, stride, mp_kernel, mp_stride, tf.nn.relu) 
    next_input = tf.nn.dropout(next_input, keep_prob, name='Dropout')
    weights_list.append(weights)
  return next_input, l2_loss, weights_list

def flatten_conv_to_fc(next_input):
  with tf.name_scope('Flatten'):
    shape      = next_input.get_shape().as_list()
    next_input = tf.reshape(next_input, [-1,shape[1]*shape[2]*shape[3]])
  return next_input

def create_fully_connected_layer(layer_name, input_tensor, l2_loss, input_dim, output_dim, activation_func):
  with tf.name_scope(layer_name):
    weights = create_weights(layer_name, [input_dim, output_dim])
    biases  = create_biases(layer_name, output_dim)
    with tf.name_scope('XW_plus_b'):
      pre_activations = tf.matmul(input_tensor, weights) + biases
      tf.histogram_summary(layer_name+'/1_pre_activations', pre_activations)
    activations = create_activations(layer_name, pre_activations, activation_func)
    l2_loss = add_l2_losses(layer_name, l2_loss, weights, biases)
  return activations, l2_loss

def create_all_fc_layers(next_input, l2_loss, fc_layers, keep_prob):
  fc_layer_count = len(fc_layers)
  input_dims     = next_input.get_shape().as_list()[1]
  fc_node_counts = [input_dims] + fc_layers
  for i in range(fc_layer_count):
    next_input, l2_loss = create_fully_connected_layer('Fully_Connected'+str(i+1), next_input, l2_loss,
                                                       fc_node_counts[i],fc_node_counts[i+1],
                                                       tf.nn.relu if i+1 < fc_layer_count else tf.identity) 
    next_input = tf.nn.dropout(next_input, keep_prob, name='Dropout') if i+1 < fc_layer_count else next_input
  return next_input, l2_loss 

def create_logits_for_all_outputs(features, l2_loss):
  input_dim = features.get_shape().as_list()[1]
  logits_list = []
  for i in range(NUM_CLASSIFIERS):
    layer_name = 'Digit_'+str(i+1)
    logits, l2_loss = create_fully_connected_layer(layer_name, features, l2_loss, input_dim, NUM_LABELS, tf.identity)
    logits_list.append(logits)
  return logits_list, l2_loss

And finally, define the functions to calculate loss and accuracy, create the training step, and any helper functions to run the model.

In [6]:
def calculate_loss(logits_list, labels, l2_loss_list, weight_decay, batch_size):
  with tf.name_scope('Loss'):
    with tf.name_scope('Cross_entropy'):
      cross_entropy_total = 0.0
      cross_entropy_list = []                 # create this list to return for debugging 
      labels_list = tf.unpack(labels, axis=1) # unpack to have one loss object per classifier in the graph
      for i in range(NUM_CLASSIFIERS):
        name            = 'Digit_'+str(i+1)
        cross_entropies = tf.nn.sparse_softmax_cross_entropy_with_logits(logits_list[i], labels_list[i])
        cross_entropy_list.append(cross_entropies) 
        cross_entropy   = tf.reduce_mean(cross_entropies, name=name)
        tf.scalar_summary('Loss/Cross_entropy/'+name, cross_entropy)
        cross_entropy_total += cross_entropy
      tf.scalar_summary('Loss/Cross_entropy/Total', cross_entropy_total)
    with tf.name_scope('L2_loss'):
      l2_loss = tf.add_n(l2_loss_list)
      l2_loss_component = weight_decay * batch_size / TRAIN_SIZE * l2_loss
      tf.scalar_summary('Parameters/Weight_decay',weight_decay)
      tf.scalar_summary('Loss/L2_loss_component',l2_loss_component)
    loss = cross_entropy_total + l2_loss_component
    tf.scalar_summary('Loss/Total',loss)
  return loss, cross_entropy_list

def calculate_accuracy(logits_list, labels):
  with tf.name_scope('Accuracy'):
    with tf.name_scope('Predictions'):
      logits_array = tf.pack(logits_list, axis=1) # put the logits into a big array: [samples, classifiers, labels]
      predictions  = tf.argmax(logits_array, 2)   # find the label with the highest logit (softmax is monotonic)
    with tf.name_scope('Accuracy'):
      correct_predictions = tf.reduce_all(tf.equal(predictions, labels),1) # sample is correct if all classifiers are correct 
      accuracy            = 100.0 * tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
    tf.scalar_summary('Accuracy',accuracy)
  return accuracy

def create_training_step(loss, initial_rate, learn_decay_steps, learn_decay_rate, algorithm, global_step):
  with tf.name_scope('Train'):
    if algorithm == 'Adam': 
      optimizer = tf.train.AdamOptimizer()
    else:
      with tf.name_scope('Learning_rate'):
        learning_rate = tf.train.exponential_decay(initial_rate, global_step, learn_decay_steps, learn_decay_rate)
        tf.scalar_summary('Parameters/Learning rate',learning_rate)
      if algorithm == 'AdLR': 
        optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
      elif algorithm[0]  == 'M': 
        optimizer = tf.train.MomentumOptimizer(learning_rate, float(algorithm[1:]))  # assume "M0.5" for momentum 0.5 
      else: 
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_step = optimizer.minimize(loss, global_step=global_step)
  return train_step

def get_next_batch(step, batch_size, train_dataset, train_labels):
  offset = (step * batch_size) % (TRAIN_SIZE - batch_size) 
  batch_data   = train_dataset[offset:(offset + batch_size), :, :, :]
  batch_labels =  train_labels[offset:(offset + batch_size), :]
  return batch_data, batch_labels

def log_bucket_delta(step, num_log_buckets, num_steps):
  return (step + 1) * num_log_buckets // num_steps - step * num_log_buckets // num_steps

**3.3 Set up the model **

Using the helper functions, create the graph according to the specified design and regularisation hyperparameters. Then create the loop to run it according to the specified batch size, iterations and logging parameters.

In [7]:
def define_and_run_model(data, params, log_params):
  # unpack the arguments
  train_dataset, train_labels, \
    test_dataset, test_labels, test_photos                = data
  num_steps, batch_size, (conv_layers, _), fc_layers,\
    initial_rate, learn_decay_rate, learn_decay_steps, \
    weight_decay, algorithm, dropout, nickname            = params
  log_dir, log_printouts, tb_printouts, tb_show_all       = log_params
  
  # set up logging parameters
  do_logging     = log_printouts > 0
  do_tensorboard = tb_printouts  > 0
  log_printouts  = min(num_steps, log_printouts) - 1 # can't log more times than we have steps, ... 
  tb_print_steps = min(num_steps, tb_printouts)  - 1 # ... and subtract 1 because the first step is always logged.
  
  # set up network constants
  global TRAIN_SIZE, IMAGE_SIZE, NUM_CHANNELS, NUM_LABELS, NUM_CLASSIFIERS, TENSORBOARD_SHOW_ALL
  TENSORBOARD_SHOW_ALL = tb_show_all
  TRAIN_SIZE = train_dataset.shape[0]
  graph = tf.Graph()

  # define the graph
  with graph.as_default():
   
    # create placeholders and variables
    input_data, labels, keep_prob = create_placeholders()
    global_step = tf.Variable(0, trainable=False, name='Global_step')

    # build the network
    next_input,  l2_loss = input_data, []
    next_input,  l2_loss, conv_weights = create_all_conv_layers(next_input, l2_loss, conv_layers, keep_prob)
    next_input           = flatten_conv_to_fc(next_input)
    features,    l2_loss = create_all_fc_layers(next_input, l2_loss, fc_layers, keep_prob)
    logits_list, l2_loss = create_logits_for_all_outputs(features, l2_loss)
    
    # calculate loss and accuracy; set up the training step
    loss, loss_list = calculate_loss(logits_list, labels, l2_loss, weight_decay, batch_size)
    accuracy        = calculate_accuracy(logits_list, labels)
    train_step      = create_training_step(loss, initial_rate, learn_decay_steps, learn_decay_rate, algorithm, global_step)

    # bring together the summaries, create writers for them
    merged = tf.merge_all_summaries()

  # now run the graph
  with tf.Session(graph=graph) as sess:

    # initialise the summary writers
    train_writer = tf.train.SummaryWriter(log_dir+'/train', graph)
    test_writer  = tf.train.SummaryWriter(log_dir+'/test')

    # initialise log and timer for the homemade plotter
    learning_log = np.array([]).reshape(0,5) # the array we'll use to plot the learning curve
    start = timer()

    # the test and photo feed dictionary doesn't change so define it now
    test_feed_dict  = {input_data : test_dataset, labels : test_labels, keep_prob : 1.0}
    photo_feed_dict = {input_data : test_photos, keep_prob : 1.0}
  
    tf.initialize_all_variables().run()

    for step in range(num_steps):

      # define the feed dictionary for the training run - this happens every step
      batch_data, batch_labels = get_next_batch(step, batch_size, train_dataset, train_labels)
      train_feed_dict = {input_data : batch_data, labels : batch_labels, keep_prob : dropout}

      # sometimes we'll calculate test accuracy and save to tensorboard
      if do_tensorboard and (step == 0 or log_bucket_delta(step, tb_printouts, num_steps) > 0):
        _, train_loss, train_acc, train_summary = sess.run([train_step, loss, accuracy, merged], feed_dict=train_feed_dict)
        test_loss,      test_acc,  test_summary = sess.run([            loss, accuracy, merged], feed_dict=test_feed_dict)
        train_writer.add_summary(train_summary, step)
        test_writer.add_summary( test_summary,  step)

      # sometime we'll just calculate test accuracy for the homemade plotter
      elif do_logging and (step == 0 or log_bucket_delta(step, log_printouts, num_steps) > 0):
        _, train_loss, train_acc = sess.run([train_step, loss, accuracy], feed_dict=train_feed_dict)
        test_loss, test_acc      = sess.run([            loss, accuracy], feed_dict=test_feed_dict)

      # and otherwise, we just train the network
      else: sess.run([train_step], feed_dict=train_feed_dict)

      if do_logging and log_bucket_delta(step, log_printouts, num_steps) > 0 :
        learning_log = np.append(learning_log, [[step, train_loss, test_loss, train_acc, test_acc]], axis=0)
        
      dots_to_print = log_bucket_delta(step, 100, num_steps)
      if dots_to_print > 0: print('.'*dots_to_print, end='')

    # Finally, close the writers,  calculate accuracy on the test data:
    train_writer.close()
    test_writer.close()
    test_accuracy, test_logits, test_losses, c_weights = sess.run([accuracy, logits_list, loss_list, conv_weights],
                                                        feed_dict=test_feed_dict)
    photo_logits = sess.run([logits_list], feed_dict = photo_feed_dict)
    run_time     = timer() - start
    output_data  = test_logits, test_losses, photo_logits, c_weights
  return learning_log, run_time, test_accuracy, output_data

**3.4 Run the model with different hyperparameters**

First, set up any helper functions for the logging, and load the saved results log and set.

In [8]:
run_text_header = 'run  step btc conv_layers  fc_layers   learnR  ldr  lds     wd   alg d/o Nickname         - t_acc min'
     # e.g.        85: 10000 123 cl_16_32_128 [1024]        0.05 0.95 1000  0.001 M0.05 1.0 L_r_decay 0.9/k  - 87.4%   8
     # sizes        2:     5   3 12           11               6    4    4      6     5   3 16               -    4%   3

def run_text_long(params, run_counter, test_accuracy, run_time): # get the whole config printable to one line
  st, b, (_, conv_l), fc_l, lr, ldr, lds, wd, l_a, do, Nickname = params
  return '{:>2d}: {:>5d} {:>3d} {:<12} {:<11} {:>6} {:>4} {:>4} {:>6} {:>5} {:>3.1f} {:<16} - {:>4.1f}% {:>3.0f}'.format(
          run_counter, st, b,    conv_l, fc_l,   lr,   ldr, lds,   wd,  l_a,  do, Nickname, test_accuracy, run_time/60)

def run_text_short(results):
  i, params, _, _, test_accuracy, run_time, _ = results
  return '{:>2d}: {:<20} - {:>4.1f}% acc in {:>4.0f}m'.format(i, params[-1], test_accuracy, run_time/60)
  
def load_results_log(filename):
  if os.path.exists(filename):
    with open(filename, 'rb') as f:
      results_log = pickle.load(f)
    run_counter = len(results_log) 
    print('Results log loaded from {}'.format(filename))
  else:
    results_log, run_counter = [],0
    print('Results log initialised')
  return results_log, run_counter

def save_results_log(data, filename):
  pickle_file = filename
  os.environ['TZ'] = 'Europe/Zurich'
  time.tzset()
  try:
    f = open(pickle_file, 'wb')
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
    print("{} saved at {}".format(filename,strftime("%H:%M")))
    f.close()
  except Exception as e:
    print('Unable to save data to', filename, ':', e)
    raise

if 'results_log' not in locals() or True:
  results_log, run_counter = load_results_log("results.pickle")

Results log loaded from results.pickle


Now specify a list of hyperparameter combinations and run the model on those, adding the results to the results log.

In [9]:
########################## Set the hyperparameters to test #############################
# conv_layers = ( [(patch_size, depth, stride, mp_kernel, mp_stride), ...] , name)     #
# params = 0 steps, 1 batch, 2 convolution_layers, 3 fc_layers,                        #
#          4 learn_rate, 5 ldecay_rate, 6 ldecay_steps,                                #
#          7 weight_decay, 8 learning algorithm, 9 dropout, 10 Nickname                #
########################################################################################

def define_conv_layers(node_list, stride_list, name=None):
  layer = [(5, nodes, stride, 2, 2) for nodes, stride in zip(node_list, stride_list)]
  if name is None: name = 'cl_'+'_'.join(str(node) for node in node_list)
  return (layer, name)

c2_16_32s2   = define_conv_layers([16,32],        [2,2],     'c2_16>32s2')
c3_16_64s121 = define_conv_layers([16,32,64],     [1,2,1],   'c3_16>64s121')
c3_16_64s212 = define_conv_layers([16,32,64],     [2,1,2],   'c3_16>64s212')
c3_32_64s121 = define_conv_layers([32,32,64],     [1,2,1],   'c3_32>64s121')
c3_32_64s212 = define_conv_layers([32,32,64],     [2,1,2],   'c3_32>64s212')
c4_16_128s12 = define_conv_layers([16,32,64,128], [1,1,1,2], 'c4_16>128s12')

#  0 st, 1 b,    2 conv_l,      3 fc_l,   4 lr, 5 ldr, 6 lds,  7 wd,  8 l_a, 9 do, 10 Nickname     |
param_list = [ \
 (10000, 128,c4_16_128s12,  [256, 128],   0.001 ,  0.92,   500, 0.01 , "AdLR",  0.9, "Final model")]

log_printouts =   100            # How many times to write validation stats to the log file 
tb_printouts  =    20            # How many times to write train & validation stats to tensorboard
tb_show_all   = False            # Whether to log stats on the weights and biases
log_path      = "tb/optimise/"   # Where to save the tensorboard logs
progress_bar  = '    |' + '|'.join('{:<2}       '.format(str(i*10)) for i in range(10)) + '|100'
data          = (train_dataset, train_labels, test_dataset, test_labels, test_photos)

print(run_text_header)
print(progress_bar)

##### Run the model and log the results #####
for params in param_list:
  log_dir    = log_path + str(run_counter) + "_" + params[-1]
  log_params = (log_dir, log_printouts, tb_printouts, tb_show_all)

  print('{:>2}: .'.format(run_counter), end='')
  learning_log, run_time, test_accuracy, output_data = define_and_run_model(data, params, log_params)
  run_text = run_text_long(params, run_counter, test_accuracy, run_time)
  print('\n' + run_text)
  results_log.append((run_counter, params, run_text, learning_log, test_accuracy, run_time, output_data))
  run_counter += 1

save_results_log(results_log, 'results.pickle')

run  step btc conv_layers  fc_layers   learnR  ldr  lds     wd   alg d/o Nickname         - t_acc min
    |0        |10       |20       |30       |40       |50       |60       |70       |80       |90       |100
23: .....................................................................................................
23: 10000 128 c4_16>128s12 [256, 128]   0.001 0.92  500   0.01  AdLR 0.9 Final model      - 91.3% 541
results.pickle saved at 07:18


**3.5 Show the results**

Show the results in detail in TensorBoard.

In [ ]:
# !sudo netstat -ap | grep :8889    # in case Tensorboard hangs, use this to find the process to kill...
# !kill 1014                        # ... and kill it.
!tensorboard --port 8889 --logdir tb/optimise/

Starting TensorBoard 28 on port 8889
(You can navigate to http://172.17.0.2:8889)


List the runs saved to the results log.

In [11]:
print(run_text_header)
for n,results in enumerate(results_log): 
  i, params, run_text, learning_log, test_accuracy, run_time, output_data = results
  print(run_text)

run  step btc conv_layers  fc_layers   learnR  ldr  lds     wd   alg d/o Nickname         - t_acc min
 0: 10000 128 c2_16>32s2   [1024]        None  1.0    1    0.0  Adam 1.0 Baseline         - 48.3%  60
 1: 10000 128 c2_16>32s2   [1024]       0.001  1.0    1    0.0  AdLR 1.0 AdamLR=0.001     - 49.2%  60
 2: 10000 128 c2_16>32s2   [1024]      0.0001  1.0    1    0.0  AdLR 1.0 AdamLR=0.0001    - 31.9%  59
 3: 10000 128 c2_16>32s2   [1024]       0.001  0.9 1000    0.0  AdLR 1.0 AdamLRD 0.9/k    - 48.1%  61
 4: 10000 128 c2_16>32s2   [256]         None  1.0    1    0.0  Adam 1.0 Only 256FCnodes  - 49.0%  55
 5: 10000 128 c2_16>32s2   [256]        0.001 0.97  100    0.0  AdLR 1.0 LRD 0.97/100     - 45.3%  56
 6: 10000 128 c2_16>32s2   [256]         None  1.0    1   0.01  Adam 1.0 W_decay 0.01     - 47.8%  54
 7: 10000 128 c2_16>32s2   [256]         None  1.0    1  0.001  Adam 1.0 W_decay 0.001    - 48.3%  54
 8: 10000 128 c2_16>32s2   [256, 64]     None  1.0    1    0.0  Adam 1.0 2 FC laye